# GHS Data Download and Aggregating

This notebook describes the process of downloading and aggregating GHS data from [here](https://ghsl.jrc.ec.europa.eu/download.php?). The notebook contains aggregation for the grid:

- Classification of pixels as rural or urban for the raster.
    - Using 21 and above for urban and 13 and below for rural.
- Fraction of each grid pixel that is rural/ urban
- Find population by grid pixel.

The epoch used in closest to the date `27th Feb 2023` and would be 2025.

In [1]:
%load_ext jupyter_black
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
import requests, zipfile, io
import rasterio
import rasterio.mask

In [3]:
base_dir = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_dir / "06_settlement/input/"
shp_input_dir = base_dir / "02_housing_damage/input/"
grid_input_dir = base_dir / "02_housing_damage/output/"
output_dir = base_dir / "06_settlement/output/"

In [4]:
adm3_shp = gpd.read_file(
    shp_input_dir / "phl_adminboundaries_candidate_adm3.zip"
)

# grid
grid = gpd.read_file(grid_input_dir / "phl_0.1_degree_grid_land_overlap.gpkg")

In [5]:
smod_link = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_SMOD_GLOBE_R2022A/GHS_SMOD_P2025_GLOBE_R2022A_54009_1000/V1-0/GHS_SMOD_P2025_GLOBE_R2022A_54009_1000_V1_0.zip"
pop_link = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2022A/GHS_POP_P2025_GLOBE_R2022A_54009_100/V1-0/tiles/"

## Degree of Urbanisation

### Urban-Rural Classification

In [ ]:
# Downloading the whole global data set as it is small
req = requests.get(smod_link, verify=False, stream=True)
with zipfile.ZipFile(io.BytesIO(req.content)) as zObj:
    fileNames = zObj.namelist()
    for fileName in fileNames:
        if fileName.endswith("tif"):
            content = zObj.open(fileName).read()
            open(input_dir / "SMOD" / fileName, "wb").write(content)

In [6]:
# Reading in raster
file_name = os.listdir(input_dir / "SMOD")
smod_raster = rasterio.open(input_dir / "SMOD" / file_name[0])
smod_array = smod_raster.read(1)
smod_array

array([[-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       ...,
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200]], dtype=int16)

In [7]:
# no data are set at -200
# water seems to be set to 10
# converting to similar CRS
smod_raster.crs

CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [8]:
grid.total_bounds

array([114.25,   4.55, 126.65,  21.15])

In [ ]:
# converting crs and clipping
# converting to CRS similar to grid will cause some distortion

polygons = [feature["geometry"] for feature in adm3_shp]
rasterio.mask.mask(smod_raster, polygons, crop=True)

In [ ]:
# converting to CRS similar to grid will cause some distortion
out_meta = smod_raster.meta.copy()
out_meta.update(
    {
        "driver": "GTiff",
        "height": smod_raster.shape[1],
        "width": smod_raster.shape[2],
        "transform": smod_raster,
        "crs": grid.crs
    }
)
with rasterio.open(input_dir / "PHL_GHS_SMOD_P2025_GLOBE_R2022A_54009_1000_V1_0.tif", "w", **out_meta) as dest:
    dest.write(smod_raster)
phl_smod_raster = rasterio.open(input_dir / "SMOD" / "PHL_GHS_SMOD_P2025_GLOBE_R2022A_54009_1000_V1_0.tif")
phl_smod_array = phl_smod_raster.read(1)

## Population

### Total Population by grid